<i><h2 style="font-family:serif;font-size:250%; text-align:center;color:#004d80"> Tarea III: Neo4j</h2></i>
<i><h3 style="font-family:serif;font-size:100%; text-align:center;color:#FFFFF"> Jorge Caullán </h3></i></i>
<i><h3 style="font-family:serif;font-size:100%; text-align:center;color:#FFFFF"> Miguel Huichaman </h3></i></i>
<i><h3 style="font-family:serif;font-size:100%; text-align:center;color:#FFFFF"> Ignacio Loayza</h3></i></i>

# Introducción

A continuación se presenta el análisis de dos conjuntos de datos almacenados en el motor de base de datos `Neo4j`.
Los dos conjuntos a analizar corresponden a la red del metro de Santiago de Chile, donde los nodos corresponden a las estacione y las relaciones entre nodos son hechas en base a la conexión directa entras las estaciones.



### inicialización de la base de datos
nota: si se tiene problemas para conectar a neo4j desde el navegador probar
* host: bolt://localhost:7687
* usuario: neo4j
* password: neo4j

1) si funciona, pedira cambiar el pass: poner cualquier cosa y usar como comando en el navegador
    * :server change-password
    * despues volver al password neo4j
   
2) si hay problemas, usar desde la consola donde esta el docker:
    * docker-compose exec neo4j bash
    * neo4j-admin set-initial-password neo4j

    y reintentar lo anterior

librerias requeridas:
* pip install py2neo

In [28]:
from py2neo import Graph, Node, Relationship, NodeMatcher

# conectarse
graph = Graph("bolt://localhost:7687", auth=("neo4j", "Diegoql2"))

graph.delete_all()

crear nodos

In [2]:
# nicole = Node("Person", name="Nicole", age=24)
# drew = Node("Person", name="Drew", age=20)

# mtdew = Node("Drink", name="Mountain Dew", calories=9000)
# cokezero = Node("Drink", name="Coke Zero", calories=0)

# coke = Node("Manufacturer", name="Coca Cola")
# pepsi = Node("Manufacturer", name="Pepsi")

# graph.create(nicole | drew | mtdew | cokezero | coke | pepsi)

crear relaciones

In [3]:
# graph.create(Relationship(nicole, "LIKES", cokezero))
# graph.create(Relationship(nicole, "LIKES", mtdew))
# graph.create(Relationship(drew, "LIKES", mtdew))
# graph.create(Relationship(coke, "MAKES", cokezero))
# graph.create(Relationship(pepsi, "MAKES", mtdew))

obtener data (por una query con parametros)

In [4]:
# query = """
# MATCH (person:Person)-[:LIKES]->(drink:Drink)
# WHERE person.name = {nicole_name}
# RETURN person.name AS name, drink.name AS drink
# """

# data = graph.run(query, nicole_name='Nicole')

# for d in data:
#     print(d)

## I) Enrutamiento

> ### Ingestión de los datos hacia la base de datos

In [29]:
import json
with open('Santiago_de_Chile.json', encoding='utf-8') as fh:
    metros = json.load(fh)

matcher = NodeMatcher(graph)

for metro in metros:
    current_station_node = matcher.match("Station", name=metro['station']).first()
    if current_station_node:
        if current_station_node['line'] != metro['line']:
            current_station_node['line'] = '{},{}'.format(current_station_node['line'], metro['line'])
            graph.push(current_station_node)
    else:
        current_station_node = Node("Station", name=metro['station'], line=metro['line'])
        graph.create(current_station_node)

    if metro['previous'] != None:
        previous_node = matcher.match("Station", name=metro['previous']['station']).first()
        graph.create(Relationship(previous_node, "CONNECT", current_station_node, distance=metro['previous']['distance_in_meters']))
    if metro['next'] != None:
        next_node = matcher.match("Station", name=metro['next']['station']).first()
        if next_node:
            graph.create(Relationship(current_station_node, "CONNECT", next_node, distance=metro['next']['distance_in_meters']))

> ### 1) Seleccione y muestre todas las estaciones correspondientes a una lı́nea de metro (Excluyendo lı́nea 4A).

En este caso, recuperaremos todas las estaciones correspondientes a la linea `L4` que además no estén en la línea ´L4A´:

In [59]:

required_line = 'L4'


query = """
MATCH (n)
WHERE n.line contains {required_line} AND (not(n.line contains 'L4A'))
RETURN n
"""

data = graph.run(query, required_line=required_line)
print('Estaciones en {} y no en L4:\n'.format(required_line))
for d in data:
    print(dict(d)['n']['name'])

Estaciones en L4 y no en L4:

Plaza Egaña
Cristóbal Colón
Francisco Bilbao
Príncipe de Gales
Simón Bolívar
Los Orientales
Grecia
Los Presidentes
Quilín
Las Torres
Macul
Vicente Valdés
Rojas Magallanes
Trinidad
San José de la Estrella
Los Quillayes
Elisa Correa
Hospital Sótero del Río
Protectora de la Infancia
Las Mercedes
Plaza de Puente Alto
Tobalaba


> ### 2) Muestre las 10 estaciones mas cercanas entre sí

Primero, se recuperan todos aquellos objetos que conectan con una estación, luego, se filtra solo aquellos atributos relevantes, en nuestro caso, `name` de la primera estación, `distance` que representa el peso de la arista entre las dos estaciones y `name` de la segunda estación. Finalmente, se ordena según `distance` (orden creciente) y se limita el retorno de la consulta a solo 10 elementos:

In [70]:
query = """
MATCH(first)-[c:CONNECT]->(second:Station)
RETURN first.name, c.distance, second.name 
ORDER BY c.distance 
LIMIT 10
"""

data = graph.run(query)

print('Las estaciones más cercanas entre sí son:\n')
for i, d in enumerate(data):
    data_dict = dict(d)
    print('{3}. {0} y {1}, separadas por un distancia de {2}'.format(data_dict['first.name'],
                                                                data_dict['second.name'],
                                                                data_dict['c.distance'],
                                                                i+1))

Las estaciones más cercanas entre sí son:

1. La Moneda y Universidad de Chile, separadas por un distancia de 400
2. Los Héroes y La Moneda, separadas por un distancia de 460
3. Universidad de Chile y Santa Lucía, separadas por un distancia de 480
4. Parque Bustamante y Santa Isabel, separadas por un distancia de 500
5. Las Rejas y Ecuador, separadas por un distancia de 520
6. Santa Lucía y Universidad Católica, separadas por un distancia de 530
7. Carlos Valdovinos y Camino Agrícola, separadas por un distancia de 540
8. San Alberto Hurtado y Universidad de Santiago, separadas por un distancia de 540
9. Pedro de Valdivia y Los Leones, separadas por un distancia de 550
10. Unión Latinoamericana y República, separadas por un distancia de 560


> ### 3) Busque la ruta mas corta entre una estacion de metro y la estacion Camino Agricola, esta ruta debe tener al menos 10 nodos y una combinacion

In [9]:
query = """
MATCH (start:Station{name:"Manuel Montt"}), (end:Station{name:"Camino Agrícola"})
CALL algo.shortestPath.stream(start, end, "distance")
YIELD nodeId, cost
RETURN algo.asNode(nodeId).name AS name, cost
""" #esta debiera ser la query, pero hay q instalar los algoritmos a neo4j y no logre hacerlo

data = graph.run(query)

for d in data:
    print(d)

ClientError: ProcedureNotFound: There is no procedure with the name `algo.shortestPath.stream` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.

* Considere que ya no puede realizar esa combinacion, busque la siguiente ruta mas corta

In [ ]:
# query = """
# MATCH (start:Station{name:"Manuel Montt"}), (end:Station{name:"Camino Agrícola"})
# CALL algo.shortestPath.stream(start, end, "distance")
# YIELD nodeId, cost
# RETURN algo.asNode(nodeId).name AS name, cost
# """

## Centralidad y Comunidades

In [3]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4j"))
# graph.delete_all()

matcher = NodeMatcher(graph)
c=0

f = open('../soc-redditHyperlinks-title.tsv', 'r')
# f = open('../soc-redditHyperlinks-body.tsv', 'r')
f.readline() #para saltar la primera linea
all_lines = f.readlines()
f.close()

# total_lines = len(all_lines)
total_lines = 10000
post_ids = []

for row in all_lines:
    if c % (total_lines/100) == 0:
        print("progress:", c/float(total_lines)*100)
        
    #cada row es una referencia entre 2 subredits
    #el post id se puede repetir y las propeties son las mismas cuando se repiten
    row = row.replace('\n','').split('\t')
    if row[2] in post_ids:
        positive = 0
        negative = 0
    else:
        post_ids.append(row[2])
        if row[4] == '1':
            positive = 1
            negative = 0
        else:
            positive = 0
            negative = 1
    
    #crear subreddits si no existen
    subreddit_node = matcher.match("Subreddit", name=row[0]).first()
    if subreddit_node: #agregar el sentimiento del post al subredit
        subreddit_node['positives'] = subreddit_node['positives']+positive
        subreddit_node['negatives'] = subreddit_node['negatives']+negative
        graph.push(subreddit_node)
    else:
        subreddit_node = Node("Subreddit", name=row[0], positives=positive, negatives=negative)
        graph.create(subreddit_node)
        
    subreddit_hiperlink_node = matcher.match("Subreddit", name=row[1]).first()
    if not subreddit_hiperlink_node:
        subreddit_hiperlink_node = Node("Subreddit", name=row[1], positives=0, negatives=0)
        graph.create(subreddit_hiperlink_node)
    
    #relacion entre ambos subreddits
    graph.create(Relationship(subreddit_node, "REFERENCE", subreddit_hiperlink_node))
    
    #esto hacia que se demorara mucho el paso de la data, ademas de no poder distinguir nada en el grafo final
#     #crear post si no existe
#     post_node = matcher.match("Post", id=row[2]).first()
#     if not post_node:
#         properties = row[5].split(',')
#         post_node = Node("Post", id=row[2], sentiment=row[4], characters=properties[0], words=properties[7])
    
#     #relacion entre post y subredit de origen
#     graph.create(Relationship(subreddit_node, "HAVE_POST", post_node))
    
    c+=1
    if c==total_lines:
        break

progress: 0.0
progress: 1.0
progress: 2.0
progress: 3.0
progress: 4.0
progress: 5.0
progress: 6.0
progress: 7.000000000000001
progress: 8.0
progress: 9.0
progress: 10.0
progress: 11.0
progress: 12.0
progress: 13.0
progress: 14.000000000000002
progress: 15.0
progress: 16.0
progress: 17.0
progress: 18.0
progress: 19.0
progress: 20.0
progress: 21.0
progress: 22.0
progress: 23.0
progress: 24.0
progress: 25.0
progress: 26.0
progress: 27.0
progress: 28.000000000000004
progress: 28.999999999999996
progress: 30.0
progress: 31.0
progress: 32.0
progress: 33.0
progress: 34.0
progress: 35.0
progress: 36.0
progress: 37.0
progress: 38.0
progress: 39.0
progress: 40.0
progress: 41.0
progress: 42.0
progress: 43.0
progress: 44.0
progress: 45.0
progress: 46.0
progress: 47.0
progress: 48.0
progress: 49.0
progress: 50.0
progress: 51.0
progress: 52.0
progress: 53.0
progress: 54.0
progress: 55.00000000000001
progress: 56.00000000000001
progress: 56.99999999999999
progress: 57.99999999999999
progress: 59.0
pr

* Encontrar los subreddits mas referenciados dentro de 3 comunidades diferentes. Que tienen en comun?(centralidad).

In [9]:
query = """
MATCH (n:Subreddit)-[:REFERENCE]->(m) 
return m, collect(n) as subsubreddits
order by size(subsubreddits) desc limit 3
"""

data = graph.run(query)

subreddit_names = []
for d in data:
    subreddit_names.append(d[0])
    print(d)

<Record m=(_147:Subreddit {name: 'askreddit', negatives: 20, positives: 124}) subsubreddits=[(_9:Subreddit {name: 'forwardsfromgrandma', negatives: 0, positives: 1}), (_37:Subreddit {name: 'lebowski', negatives: 0, positives: 1}), (_50:Subreddit {name: 'bestoftldr', negatives: 8, positives: 25}), (_52:Subreddit {name: 'switcharoo', negatives: 5, positives: 115}), (_59:Subreddit {name: 'examplesofevil', negatives: 0, positives: 33}), (_77:Subreddit {name: 'adviceof', negatives: 0, positives: 20}), (_90:Subreddit {name: 'notcirclejerk', negatives: 0, positives: 40}), (_102:Subreddit {name: 'titlegore', negatives: 0, positives: 116}), (_108:Subreddit {name: 'gmomyths', negatives: 13, positives: 23}), (_120:Subreddit {name: 'vegetarian', negatives: 0, positives: 3}), (_129:Subreddit {name: 'teenguide', negatives: 0, positives: 9}), (_143:Subreddit {name: 'funny', negatives: 1, positives: 23}), (_145:Subreddit {name: 'writingprompts', negatives: 7, positives: 54}), (_146:Subreddit {name: 'c

<img src="subreddits mas referenciados.png">

In [11]:
from pprint import pprint
pprint(subreddit_names)

[(_147:Subreddit {name: 'askreddit', negatives: 20, positives: 124}),
 (_1320:Subreddit {name: 'iama', negatives: 2, positives: 74}),
 (_1315:Subreddit {name: 'pics', negatives: 1, positives: 5})]


* Considerando el atributo POST_LABEL, estudiar un caso donde una comunidad se reera negativamente de otra.

In [13]:
query = """
match (n:Subreddit)-[:REFERENCE]->(m) 
where n.positives < n.negatives
return m, collect(n) as subr
order by size(subr) desc limit 3
"""

data = graph.run(query)

subreddit_names = []
for d in data:
    print(d)

<Record m=(_147:Subreddit {name: 'askreddit', negatives: 20, positives: 124}) subr=[(_158:Subreddit {name: 'twobestfriendsplay', negatives: 1, positives: 0}), (_268:Subreddit {name: 'muhfeelings', negatives: 3, positives: 2}), (_334:Subreddit {name: 'itcrowd', negatives: 1, positives: 0}), (_649:Subreddit {name: 'toasterrights', negatives: 2, positives: 0}), (_833:Subreddit {name: 'mynextband', negatives: 3, positives: 2}), (_1806:Subreddit {name: 'trueatheism', negatives: 4, positives: 2}), (_2062:Subreddit {name: 'divorce', negatives: 1, positives: 0}), (_2554:Subreddit {name: 'entwives', negatives: 2, positives: 1}), (_3415:Subreddit {name: 'rayromano', negatives: 1, positives: 0}), (_3732:Subreddit {name: 'singing', negatives: 1, positives: 0}), (_4380:Subreddit {name: 'mdma', negatives: 1, positives: 0}), (_4942:Subreddit {name: 'massage', negatives: 1, positives: 0}), (_5392:Subreddit {name: 'antimemewatch', negatives: 3, positives: 0}), (_5759:Subreddit {name: '19thworldproblems